# 密集检索

In [1]:
# 1. 准备模拟的数据

documents = [
    "深度学习技术在计算机视觉领域中非常重要。",
    "使用深度学习模型可以理解文档的深层语义。",
    "密集检索器的优势通过学习文档和查询的表示来提高检索的准确率。"
]

query = "密集检索的优势"

In [2]:
# 2. 加载预训练模型

from transformers import BertTokenizer, BertModel
import torch

/slurm/home/yrd/shaolab/daiyizheng/.conda/envs/langechain/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
tokenizer = BertTokenizer.from_pretrained("/slurm/home/yrd/shaolab/daiyizheng/resources/hf_weights/hfl/chinese-macbert-base")
model = BertModel.from_pretrained("/slurm/home/yrd/shaolab/daiyizheng/resources/hf_weights/hfl/chinese-macbert-base")

In [18]:
# 3. 获取向量表示
def get_embedding(text):
    # 输入预处理
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=512)
    # 获取模型最后一层隐藏状态，shape : [batch_size, sequence_length, hidden_size]
    # 原因：模型的最后一层隐藏状态通常被认为是对输入文本的一个综合表示，它融合了从文本中提取的所有关键信息。
    with torch.no_grad():
        outputs = model(**inputs)
    # 使用平均池化获取句子表示
    # print(outputs.last_hidden_state.shape)
    # print(outputs.keys())
    embeddings = outputs.last_hidden_state.mean(dim=1)
    
    return embeddings

In [19]:
# 4. 获取查询向量

query_embedding = get_embedding(query)
query_embedding.shape

odict_keys(['last_hidden_state', 'pooler_output'])


torch.Size([1, 768])

In [9]:
# 5. 获取所有文档的向量（squeeze() 压缩一维，用于后续相似性计算）[3, 1, 768]
doc_embeddings = [get_embedding(doc) for doc in documents]

In [10]:
doc_embeddings[0].shape

torch.Size([1, 768])

In [11]:
doc_embeddings = torch.stack(doc_embeddings)
doc_embeddings.shape

torch.Size([3, 1, 768])

In [12]:
doc_embeddings = doc_embeddings.squeeze()
doc_embeddings.shape

torch.Size([3, 768])

In [13]:
# 6. 计算相似度

from sklearn.metrics.pairwise import cosine_similarity

simialrities = cosine_similarity(query_embedding.numpy(), doc_embeddings.numpy())

In [20]:
final_results = simialrities[0]

print(final_results)

[0.76556075 0.81379014 0.8646308 ]


## 知识补充：平均池化
### 1. 示例句子
假设我们的句子是：“我爱自然语言处理”，并且我们已经通过一个预训练模型处理了这句话，为句子中的每个词生成了一个向量表示。假设模型的隐藏层大小为`hidden_size = 4`，我们得到以下表示（每个词的表示是一个4维向量）：

- “我” -> [0.5, 0.1, -0.4, 0.3]
- “爱” -> [0.4, -0.2, 0.6, 0.1]
- “自然” -> [0.3, 0.3, -0.1, 0.2]
- “语言” -> [-0.1, 0.4, 0.5, -0.2]
- “处理” -> [0.2, -0.3, 0.2, 0.4]

### 2. 平均池化的过程
现在，我们需要将这些词向量合并成一个单一的句子级别向量。使用平均池化的方法，我们将对每个维度分别计算平均值。具体来说，对于每个维度，我们将相应维度的所有值加起来，然后除以词的数量（在这个例子中是5）。

- 第一维度的平均值：(0.5 + 0.4 + 0.3 - 0.1 + 0.2) / 5 = 0.26
- 第二维度的平均值：(0.1 - 0.2 + 0.3 + 0.4 - 0.3) / 5 = 0.06
- 第三维度的平均值：(-0.4 + 0.6 - 0.1 + 0.5 + 0.2) / 5 = 0.16
- 第四维度的平均值：(0.3 + 0.1 + 0.2 - 0.2 + 0.4) / 5 = 0.16

因此，通过平均池化，我们得到句子的向量表示为[0.26, 0.06, 0.16, 0.16]。

### 3. 使用平均池化的原因
1. **统一表示**：通过平均池化，无论句子有多长，我们都可以得到一个固定大小的向量表示。这对于后续的处理非常重要，因为它允许我们使用固定大小的输入进行各种计算和比较。

2. **整体信息**：平均池化考虑了句子中所有词的信息，提供了一个整体的句子表示。

3. **简单有效**：平均池化是一种简单的操作，计算成本低，但在许多任务中效果却意外地好。
